In [26]:
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Text, Button, Output, VBox, HBox, Label
import ipywidgets as widgets
import numpy as np

# Create output widgets for the plot and node details
plot_output = Output()
details_output = Output()

# Initialize the graph
G = nx.Graph()

# Function to create the initial spoke-and-node network
def create_spoke_and_node_network(num_spokes, num_nodes_per_spoke):
    global G, num_spokes_global, num_nodes_per_spoke_global
    G.clear()

    num_spokes_global = num_spokes
    num_nodes_per_spoke_global = num_nodes_per_spoke

    # Add the central hub (node 0)
    G.add_node(0, pos=(0, 0), lead_time=0, name='Hub')

    # Add spokes and nodes
    for i in range(1, num_spokes + 1):
        angle = 2 * i * 3.14159 / num_spokes
        for j in range(1, num_nodes_per_spoke + 1):
            x = 10 * j * np.cos(angle)
            y = 10 * j * np.sin(angle)
            node_id = i * 100 + j  # Unique node ID
            G.add_node(node_id, pos=(x, y), lead_time=0, name=f'Spoke {i} Node {j}')
            if j == 1:
                G.add_edge(0, node_id)  # Connect the first node to the hub
            else:
                G.add_edge(node_id - 1, node_id)  # Connect subsequent nodes in the spoke

    update_plot()
    update_details_output()

# Function to update the plot
def update_plot():
    with plot_output:
        plot_output.clear_output(wait=True)
        pos = nx.get_node_attributes(G, 'pos')
        labels = {node: G.nodes[node]['name'] for node in G.nodes}
        lead_times = [G.nodes[node]['lead_time'] for node in G.nodes]
        node_sizes = [300 + 50 * lt for lt in lead_times]  # Base size plus 50 times the lead time
        plt.figure(figsize=(10, 10))
        nx.draw(G, pos, labels=labels, with_labels=True, node_size=node_sizes, node_color='skyblue', font_size=10, font_color='black', font_weight='bold', edge_color='gray')
        plt.title('Spoke-and-Node Supply Chain Network')
        plt.show()

# Function to update node details
def update_node_details(spoke_num, node_num, lead_time, name):
    node_id = spoke_num * 100 + node_num
    if node_id in G.nodes:
        G.nodes[node_id]['lead_time'] = lead_time
        G.nodes[node_id]['name'] = name
        update_plot()
        update_details_output()

# Function to update the details output
def update_details_output():
    with details_output:
        details_output.clear_output()
        for node_id, data in G.nodes(data=True):
            print(f"Node {node_id}: Name = {data['name']}, Lead Time = {data['lead_time']}")

# Function to display the current details of the selected node
def display_node_details(spoke_num, node_num):
    node_id = spoke_num * 100 + node_num
    if node_id in G.nodes:
        node_details.value = f"Node {node_id}: Name = {G.nodes[node_id]['name']}, Lead Time = {G.nodes[node_id]['lead_time']}"
    else:
        node_details.value = "Node not found."

# Interactive widgets to control the number of spokes and nodes per spoke
spoke_slider = IntSlider(min=1, max=20, step=1, value=8, description='Number of Spokes')
node_slider = IntSlider(min=1, max=10, step=1, value=3, description='Nodes per Spoke')

# Button to create the initial network
create_button = Button(description="Create Network")

# Interactive controls for node details
spoke_num_slider = IntSlider(min=1, max=20, step=1, value=1, description='Spoke #')
node_num_slider = IntSlider(min=1, max=10, step=1, value=1, description='Node #')
lead_time_slider = IntSlider(min=0, max=100, step=1, value=0, description='Lead Time')
name_text = Text(value='', description='Name')
node_details = Label(value="")

# Button to update node details
update_node_button = Button(description="Update Node")
update_hub_button = Button(description="Update Hub")

# Event handler for create button
def on_create_button_clicked(b):
    create_spoke_and_node_network(spoke_slider.value, node_slider.value)

# Event handler for update node button
def on_update_node_button_clicked(b):
    update_node_details(spoke_num_slider.value, node_num_slider.value, lead_time_slider.value, name_text.value)

# Event handler for update hub button
def on_update_hub_button_clicked(b):
    update_node_details(0, 0, lead_time_slider.value, name_text.value)

# Event handler for spoke_num_slider and node_num_slider
def on_spoke_or_node_slider_changed(change):
    display_node_details(spoke_num_slider.value, node_num_slider.value)

create_button.on_click(on_create_button_clicked)
update_node_button.on_click(on_update_node_button_clicked)
update_hub_button.on_click(on_update_hub_button_clicked)
spoke_num_slider.observe(on_spoke_or_node_slider_changed, names='value')
node_num_slider.observe(on_spoke_or_node_slider_changed, names='value')

# Display interactive widgets
ui = VBox([
    HBox([spoke_slider, node_slider, create_button]),
    HBox([Label('Update Node:')]),
    HBox([spoke_num_slider, node_num_slider, lead_time_slider, name_text, update_node_button]),
    node_details,
    HBox([Label('Update Hub:')]),
    HBox([lead_time_slider, name_text, update_hub_button]),
    plot_output,
    details_output
])

display(ui)

# Create the initial plot
create_spoke_and_node_network(spoke_slider.value, node_slider.value)
